# PISA 2022 Amazon SageMaker XGBoost
Supervised Learning with Gradient Boosted Trees: A Binary Prediction Problem With Unbalanced Classes

More info on SageMaker Immersion Day: [Workshop Link](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab2-model-training/pro-code)


### ***Change country name below!***

In [1]:
country_name = 'United_States'

In [2]:
country_name_edited = country_name.replace("_", "-")

In [3]:
# cell 02
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/xgboost-'+country_name_edited+'-new-run2'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Now let's bring in the Python libraries that we'll use throughout the analysis

In [4]:
# cell 03
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

#### Download PISA 2022 Prepared Dataset

This is our dataset output from our cleaned notebook [here](https://7z4vtvpqcoxouiu.studio.us-west-2.sagemaker.aws/jupyterlab/default/lab/tree/RTC%3Amids-capstone/notebooks/eda/Data_merging.ipynb)


In [5]:
%%time 

# cell 06

# Define local file path
local_file_path = "new_PISA_cleaned_dataset.csv"  # Change as needed

# Define S3 details
bucket_name = "sagemaker-us-west-2-986030204467"
file_key = "capstone/testfiles/new_PISA_cleaned_dataset.csv"

# Check if the file exists locally
if os.path.exists(local_file_path):
    print("📂 Loading data from local file...")
    data = pd.read_csv(local_file_path, usecols=None)
    
else:
    print("☁️ Downloading data from S3...")
    
    # Create S3 client
    s3_client = boto3.client("s3")

    # Download the file from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)

    # Read the file into pandas DataFrame
    data = pd.read_csv(response["Body"], usecols=None)

    # Save a local copy for future use
    data.to_csv(local_file_path, index=False)
    print(f"✅ File saved locally as {local_file_path}")

# Display first few rows
#data.head()

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

📂 Loading data from local file...
CPU times: user 56.7 s, sys: 7.09 s, total: 1min 3s
Wall time: 1min 8s


,CNT,CNTSCHID,CNTSTUID,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST250D06JA,ST250D07JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST251D08JA,ST251D09JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,...,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,LANGN_999,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANGN_442,LANGN_471,

#### Download dictionary for the variable names

In [6]:
# Download the file from S3
s3_client = boto3.client("s3")
dictionary_file = s3_client.get_object(Bucket=bucket_name, Key="capstone/testfiles/all_vars.csv")

# Read the file into pandas DataFrame
dictionary = pd.read_csv(dictionary_file["Body"], usecols=None)

#### Subset the data to a specific COUNTRY

In [7]:
model_data = data[data['CNT'] == country_name]
print(model_data.shape)
model_data.head()

(4552, 1127)


,CNT,CNTSCHID,CNTSTUID,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST250D06JA,ST250D07JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST251D08JA,ST251D09JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,...,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,LANGN_999,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANGN_442,LANGN_471,

#### Take out additional variables

In [8]:
# Define the list of columns to drop
columns_to_remove = ["CNTSCHID", "CNTSTUID", "OECD",
    "HOMEPOS", "RELATST", "BELONG", "BULLIED", "FEELSAFE", "SCHRISK", "PERSEVAGR", "CURIOAGR", 
    "COOPAGR", "EMPATAGR", "ASSERAGR", "STRESAGR", "EMOCOAGR", "GROSAGR", "INFOSEEK", "FAMSUP", 
    "DISCLIM", "TEACHSUP", "COGACRCO", "COGACMCO", "EXPOFA", "EXPO21ST", "MATHEFF", "MATHEF21", 
    "FAMCON", "ANXMAT", "MATHPERS", "CREATEFF", "CREATSCH", "CREATFAM", "CREATAS", "CREATOOS", 
    "CREATOP", "OPENART", "IMAGINE", "SCHSUST", "LEARRES", "PROBSELF", "FAMSUPSL", "FEELLAH", 
    "SDLEFF", "ICTRES", "FLSCHOOL", "FLMULTSB", "FLFAMILY", "ACCESSFP", "FLCONFIN", "FLCONICT", 
    "ACCESSFA", "ATTCONFM", "FRINFLFM", "ICTSCH", "ICTHOME", "ICTQUAL", "ICTSUBJ", "ICTENQ", 
    "ICTFEED", "ICTOUT", "ICTWKDY", "ICTWKEND", "ICTREG", "ICTINFO", "ICTEFFIC", "BODYIMA", 
    "SOCONPA", "LIFESAT", "PSYCHSYM", "SOCCON", "EXPWB", "CURSUPP", "PQMIMP", "PQMCAR", 
    "PARINVOL", "PQSCHOOL", "PASCHPOL", "ATTIMMP", "CREATHME", "CREATACT", "CREATOPN", 
    "CREATOR", "SCHAUTO", "TCHPART", "EDULEAD", "INSTLEAD", "ENCOURPG", "DIGDVPOL", "TEAFDBK", 
    "MTTRAIN", "DMCVIEWS", "NEGSCLIM", "STAFFSHORT", "EDUSHORT", "STUBEHA", "TEACHBEHA", 
    "STDTEST", "TDTEST", "ALLACTIV", "BCREATSC", "CREENVSC", "ACTCRESC", "OPENCUL", 
    "PROBSCRI", "SCPREPBP", "SCPREPAP", "DIGPREP", 
    "ESCS", "BMMJ1", "BFMJ2", "EFFORT1", "EFFORT2", "Option_UH", "SC209Q04JA", "SC209Q05JA", "SC209Q06JA"
]

# Drop the columns above
model_data = model_data.drop(columns=columns_to_remove, errors='ignore')  # `errors='ignore'` prevents errors if a column isn't found


In [9]:
print(model_data.shape)
model_data.head()

(4552, 1090)


,CNT,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST250D06JA,ST250D07JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST251D08JA,ST251D09JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,...,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,LANGN_999,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANGN_442,LANGN_

Amazon SageMaker's XGBoost container expects data in the libSVM or CSV data format.  **Note that the first column must be the target variable and the CSV should not include headers.**  Although repetitive, it's easiest to do this after the train|validation|test split rather than before.  This avoids any misalignment issues due to random reordering.
* `MATH_Proficient`: Is the student falling behind in Math? (Average of 10 Math plausible values < 420.07)

In [10]:
# Get percent of students not proficient in Math
proficient_n = (model_data['MATH_Proficient'] == 1).sum()
not_proficient_n = (model_data['MATH_Proficient'] == 0).sum()
not_proficient_p = round( not_proficient_n / (not_proficient_n + proficient_n) * 100, 1)
print("Students who are NOT proficient in Math: ", not_proficient_n, "(", not_proficient_p, "%)")

Students who are NOT proficient in Math:  1607 ( 35.3 %)


In [11]:
# Get imbalance ratio (used as a parameter in xgboost)
not_proficient_pp = not_proficient_n / (not_proficient_n + proficient_n)

if not_proficient_pp < 0.5:
    imbalance_ratio = (1 - not_proficient_pp) / not_proficient_pp
else:
    imbalance_ratio = not_proficient_pp / (1 - not_proficient_pp)
    
print("Imbalance ratio:", round(imbalance_ratio,1))

Imbalance ratio: 1.8


In [12]:
# Reorder columns to bring 'MATH_Proficient' first
new_order = ['MATH_Proficient'] + [col for col in model_data.columns if col != 'MATH_Proficient']
model_data = model_data[new_order]

# Check the shape after dropping
print(model_data.shape)

model_data.head()

(4552, 1090)


,MATH_Proficient,CNT,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST250D06JA,ST250D07JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST251D08JA,ST251D09JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,...,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,LANGN_999,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANGN_442,LANGN_

We'll randomly split the data into 3 uneven groups.  **The model will be trained on 70% of data, it will then be evaluated on 15% of data to give us an estimate of the accuracy we hope to have on "new" data, and 15% will be held back as a final testing dataset which will be used later on.**

A seed is included in the code so the splits can be replicated!

In [13]:
# cell 12
# Randomly sort the data then split out first 70%, second 15%, and last 15%
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.85 * len(model_data))])   

/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
print("Number of rows in FULL dataset:", model_data.shape[0])

train_data_percent = round(train_data.shape[0]/model_data.shape[0] * 100, 0)
print("Number of rows in TRAINING dataset:", train_data.shape[0], "(", train_data_percent, "% )")

validation_data_percent = round(validation_data.shape[0]/model_data.shape[0] * 100, 0)
print("Number of rows in VALIDATION dataset:", validation_data.shape[0], "(", validation_data_percent, "% )")

test_data_percent = round(test_data.shape[0]/model_data.shape[0] * 100, 0)
print("Number of rows in TEST dataset:", test_data.shape[0], "(", test_data_percent, "% )")

Number of rows in FULL dataset: 4552
Number of rows in TRAINING dataset: 3186 ( 70.0 % )
Number of rows in VALIDATION dataset: 683 ( 15.0 % )
Number of rows in TEST dataset: 683 ( 15.0 % )


#### Drop country names from the dataset

In [15]:
# cell 13
#pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
#pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)# Drop non-numeric columns (e.g., country names or IDs that are not numeric)

# Drop string variables (Country names)
non_numeric_columns = train_data.select_dtypes(exclude=['number']).columns

train_data = train_data.drop(columns=non_numeric_columns)
validation_data = validation_data.drop(columns=non_numeric_columns)
test_data = test_data.drop(columns=non_numeric_columns)

# Save train dataset 
train_data.to_csv('train.csv', index=False, header=False)

# Save validation dataset 
validation_data.to_csv('validation.csv', index=False, header=False)


In [16]:
# Training data - Saved later to S3 as CSV
print(train_data.shape)
train_data.head()

(3186, 1089)


,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST250D06JA,ST250D07JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST251D08JA,ST251D09JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,ST336Q05JA,...,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,LANGN_999,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANGN_442

In [17]:
# Validation data - Saved later to S3 as CSV
print(validation_data.shape)
validation_data.head()

(683, 1089)


,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST250D06JA,ST250D07JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST251D08JA,ST251D09JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,ST336Q05JA,...,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,LANGN_999,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANGN_442

In [18]:
# Test data - NOT SAVED TO S3
print(test_data.shape)
test_data.head()

(683, 1089)


,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST250D06JA,ST250D07JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST251D08JA,ST251D09JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,ST336Q05JA,...,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,LANGN_999,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANGN_442

Now we'll copy the file to S3 for Amazon SageMaker's managed training to pickup.

In [19]:
# cell 14
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

## Training 

At a high level, gradient boosted trees works by combining predictions from many simple models, each of which tries to address the weaknesses of the previous models.  By doing this the collection of simple models can actually outperform large, complex models.  Other Amazon SageMaker notebooks elaborate on gradient boosting trees further and how they differ from similar algorithms.

`xgboost` is an extremely popular, open-source package for gradient boosted trees.  It is computationally powerful, fully featured, and has been successfully used in many machine learning competitions.  Let's start with a simple `xgboost` model, trained using Amazon SageMaker's managed, distributed training framework.

First we'll need to specify the ECR container location for Amazon SageMaker's implementation of XGBoost.

In [20]:
# cell 15
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3, which also specify that the content type is CSV.

In [21]:
# cell 16
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

First we'll need to specify training parameters to the estimator.  This includes:
1. The `xgboost` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [22]:
# cell 17
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

In [23]:
xgb.set_hyperparameters(# seed=42,       # Random seed (turned off because we are using a different seed per iteration)  
                        seed_per_iteration=True,   # Different seed for each boosting iteration, can prevent overfitting
                        early_stopping_rounds=10,   # Stop if AUC doesn’t improve for 10 rounds
                        # scale_pos_weight=imbalance_ratio, # Helps when outcome is imbalanced (but specificity decreased)
                        objective='binary:logistic',
                        eval_metric='auc', # AUCPR is better than AUC-ROC when the outcome is not balanced (but can't use with auto tuning)
                        num_round=100,   # Number of boosting rounds for training                    
                        eta=0.05,   # Learning rate, lower value is more robust to overfitting but requires more boosting rounds
                        max_depth=5,   # Deeper trees can model more complex patterns but may overfit
                        min_child_weight=10,   # Higher value ensures leaf nodes have sufficient samples, preventing overfitting
                        gamma=4,    # Higher values make it harder to partition a leaf node, making the algorithm more conservative
                        subsample=0.8,   # Fraction of training instances to use for each boosting round, < 1 can prevent overfitting
                        alpha=5   # L1 regularization term on weights, higher value leads to more regularization                       
                        )

# xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

#### Use auto-tuning to find best hyperparameters

Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose.
For example, suppose that you want to solve a binary classification problem on this marketing dataset. Your goal is to maximize the area under the curve (auc) metric of the algorithm by training an XGBoost Algorithm model. You don't know which values of the eta, alpha, min_child_weight, and max_depth hyperparameters to use to train the best model. To find the best values for these hyperparameters, you can specify ranges of values that Amazon SageMaker hyperparameter tuning searches to find the combination of values that results in the training job that performs the best as measured by the objective metric that you chose. Hyperparameter tuning launches training jobs that use hyperparameter values in the ranges that you specified, and returns the training job with highest auc.

In [24]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'num_round': IntegerParameter(50, 300),
                         'eta': ContinuousParameter(0.01, 0.1),
                         'max_depth': IntegerParameter(3, 6),
                         'min_child_weight': IntegerParameter(5, 20),
                         'gamma': IntegerParameter(1, 10),     
                         'subsample': ContinuousParameter(0.7, 1.0),
                         'alpha': IntegerParameter(1, 10),
                         }


In [25]:
tuner = HyperparameterTuner(estimator=xgb,
                            objective_metric_name='validation:auc',
                            hyperparameter_ranges=hyperparameter_ranges,
                            max_jobs=50,  
                            max_parallel_jobs=5)

# May need to adjust number of jobs depending on budget!

In [26]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.........................................................................................................................................................!


In [27]:
# cell 26
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'Completed'

In [28]:
# cell 27
# Return the best training job name
best_training_job = tuner.best_training_job()
print("Best training job:", best_training_job)

Best training job: xgboost-250227-0532-047-36a43ceb


In [29]:
# Print out hyperparameters of BEST model

response = boto3.client('sagemaker').describe_training_job(TrainingJobName=best_training_job)
best_hyperparameters = response["HyperParameters"]

best_num_round = int(best_hyperparameters["num_round"])
best_eta = float(best_hyperparameters["eta"])
best_max_depth = int(best_hyperparameters["max_depth"])
best_min_child_weight = int(best_hyperparameters["min_child_weight"])
best_gamma = int(best_hyperparameters["gamma"])
best_subsample = float(best_hyperparameters["subsample"])
best_alpha = int(best_hyperparameters["alpha"])

print("BEST num_round: ", best_num_round)
print("BEST eta: ", round(best_eta, 2))
print("BEST max_depth: ", best_max_depth)
print("BEST min_child_weight: ", best_min_child_weight)
print("BEST gamma: ", best_gamma)
print("BEST subsample: ", round(best_subsample, 2))
print("BEST alpha: ", best_alpha)


BEST num_round:  155
BEST eta:  0.07
BEST max_depth:  4
BEST min_child_weight:  13
BEST gamma:  4
BEST subsample:  0.83
BEST alpha:  1


## Deploy the model (the best model identified by HyperparameterTuner)

In [30]:
# cell 28
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')


2025-02-27 05:44:39 Starting - Found matching resource for reuse
2025-02-27 05:44:39 Downloading - Downloading the training image
2025-02-27 05:44:39 Training - Training image download completed. Training in progress.
2025-02-27 05:44:39 Uploading - Uploading generated training model
2025-02-27 05:44:39 Completed - Resource retained for reuse
------!

In [31]:
# cell 29
# Create a serializer
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batches to CSV string payloads (notice, we drop the target variable from our dataset first)
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [32]:
# Define function for predictions
def predict(data, predictor, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])
    return np.fromstring(predictions[1:], sep=',')


In [33]:
# Make predictions - Probabilities
predictions = predict(test_data.drop(['MATH_Proficient'], axis=1).to_numpy(),tuner_predictor)

In [34]:
# Clean up
tuner_predictor.delete_endpoint(delete_endpoint_config=True)

## Explain the trained model using Clarify

In [35]:
from datetime import datetime

session = sagemaker.Session()

model_name = "Clarify-{}-{}".format(country_name_edited, datetime.now().strftime("%d-%m-%Y-%H-%M-%S"))

best_model = sagemaker.estimator.Estimator.attach(best_training_job)  # Attach the best training job

model = best_model.create_model(name=model_name)  # Create a model from the best job

container_def = model.prepare_container_def()

session.create_model(model_name, role, container_def)


2025-02-27 05:44:39 Starting - Found matching resource for reuse
2025-02-27 05:44:39 Downloading - Downloading the training image
2025-02-27 05:44:39 Training - Training image download completed. Training in progress.
2025-02-27 05:44:39 Uploading - Uploading generated training model
2025-02-27 05:44:39 Completed - Resource retained for reuse


'Clarify-United-States-27-02-2025-05-49-03'

In [36]:
test_features = test_data.drop(["MATH_Proficient"], axis=1)
test_target = test_data["MATH_Proficient"]
test_features.to_csv("test_features.csv", index=False, header=False)

In [37]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.m5.2xlarge", sagemaker_session=session
)

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.large",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

In [38]:
from sagemaker.s3 import S3Downloader

# Download data from S3 to local instance
local_path = S3Downloader.download('s3://{}/{}/train'.format(bucket, prefix), './tmp/train_data')

In [39]:
# Load and sample
full_data = pd.read_csv('./tmp/train_data/train.csv', header=None)
n = min(1000, len(full_data))  # Should we decrease this? It takes 2 hours to run
sampled_data = full_data.sample(n=n)  # If full_data has less than 3000, use full sample

# Save sampled data back to S3
sampled_path = 'sampled_train_data.csv'
sampled_data.to_csv(sampled_path, index=False)

from sagemaker.s3 import S3Uploader
sampled_s3_uri = S3Uploader.upload(sampled_path, 's3://{}/{}/sampled_train'.format(bucket, prefix))

In [40]:
print(sampled_data.shape)
sampled_data.head()

(1000, 1089)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088
397,0.0,1.0,1.0,2.0,2.0,1.0,1.0,9999997,9999997,2.0,1.0,2.0,2.0,1.0,NaN,9999997,9999997,7.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,4.0,4.0,NaN,3.0,3.0,NaN,1.0,4.0,1.0,1.0,NaN,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,NaN,3.0,3.0,3.0,NaN,4.0,4.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,5.0,NaN,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,5.0,5.0,5.0,NaN,5.0,NaN,NaN,3.0,2.0,2.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.0,NaN,2.0,4.0,NaN,NaN,3.0,3.0,3.0,NaN,3.0,NaN,3.0,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,3.0,5.0,NaN,NaN,2.0,3.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2186,0.0,1.0,1.0,1.0,1.0,1.0,1.0,9999997,9999997,3.0,NaN,3.0,3.0,2.0,NaN,9999997,9999997,6.0,3.0,2.0,2.0,3.0,1.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,3.0,NaN,NaN,NaN,3.0,3.0,1.0,NaN,2.0,4.0,4.0,2.0,4.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [41]:
shap_config = clarify.SHAPConfig(
    baseline=[test_features.iloc[0].values.tolist()],
    num_samples=600,  
    agg_method="mean_abs",
    save_local_shap_values=True
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)

explainability_data_config = clarify.DataConfig(
    #s3_data_input_path='s3://{}/{}/train'.format(bucket, prefix),
    s3_data_input_path=sampled_s3_uri,
    s3_output_path=explainability_output_path,
    label='MATH_Proficient',
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

In [42]:
# Set logging level for 'sagemaker.clarify' to WARNING (hides INFO messages)
import logging

logging.getLogger("sagemaker.clarify").setLevel(logging.WARNING)

clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config
)

INFO:sagemaker:Creating processing-job with name Clarify-Explainability-2025-02-27-05-49-11-340


.................sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
We are not in a supported iso region, /bin/sh exiting gracefully with no changes.
INFO:sagemaker-clarify-processing:Starting SageMaker Clarify Processing job
INFO:analyzer.data_loading.data_loader_util:Analysis config path: /opt/ml/processing/input/config/analysis_config.json
INFO:analyzer.data_loading.data_loader_util:Analysis result path: /opt/ml/processing/output
INFO:analyzer.data_loading.data_loader_util:This host is algo-1.
INFO:analyzer.data_loading.data_loader_util:This host is the leader.
INFO:analyzer.data_loading.data_loader_util:Number of hosts in the cluster is 1.
INFO:sagemaker-clarify-processing:Running Python / Pandas based analyzer.
INFO:analyzer.data_loading.data_loader_factory:Dataset type: text/csv uri: /opt/ml/processing/input/data
INFO:sagemaker-clarify-

## Train the model again with the top 20 predictors
#### Get the list of top 20 predictors

In [43]:
# Replace with your actual bucket name and prefix used in explainability_output_path
# bucket = "your-bucket-name"
# prefix = "your-prefix"  # e.g., the folder structure used in your explainability_output_path

# Construct the S3 key for the output file
key = f"{prefix}/clarify-explainability/analysis.json"

# Initialize boto3 client for S3 and download the JSON report
s3 = boto3.client("s3")
response = s3.get_object(Bucket=bucket, Key=key)
content = response["Body"].read().decode("utf-8")
report = json.loads(content)

# Navigate to the global SHAP values dictionary
global_shap = report["explanations"]["kernel_shap"]["label0"]["global_shap_values"]

# Sort the items by the SHAP value in descending order and take the top 20
top_20 = sorted(global_shap.items(), key=lambda item: item[1], reverse=True)[:20]

# Extract just the feature names
top_20_features = [feature for feature, value in top_20]

# Print
print("Top 20 features with the highest mean absolute SHAP values:")
for feature in top_20_features:
    print(feature)


INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


Top 20 features with the highest mean absolute SHAP values:
LANGN_999
ST263Q02JA
ST259Q01JA
SC211Q03JA
ST251Q03JA
LANGN_922
ST059Q02JA
TARDYSD
ST290Q07WA
ST251Q04JA
LANGN_156
ST004D01T
IC173Q04JA
ST268Q04JA
ST251Q06JA
ST255Q01JA
IC178Q07JA
ST290Q05WA
IC174Q02JA
ST253Q01JA


In [44]:
# Make a subset of the training dataset (with only 20 predictors)
variables_to_keep = ["MATH_Proficient"] + top_20_features
train_data_small = train_data[variables_to_keep]
print(train_data_small.shape)
train_data_small.head()

(3186, 21)


,MATH_Proficient,LANGN_999,ST263Q02JA,ST259Q01JA,SC211Q03JA,ST251Q03JA,LANGN_922,ST059Q02JA,TARDYSD,ST290Q07WA,ST251Q04JA,LANGN_156,ST004D01T,IC173Q04JA,ST268Q04JA,ST251Q06JA,ST255Q01JA,IC178Q07JA,ST290Q05WA,IC174Q02JA,ST253Q01JA
575474,0.0,0,4.0,5.0,53.0,2.0,0,8.0,0.0,1.0,2.0,1,1.0,1.0,2.0,1.0,2.0,1.0,NaN,4.0,7.0
574994,1.0,0,2.0,3.0,41.0,2.0,0,NaN,2.0,NaN,2.0,0,1.0,6.0,3.0,4.0,3.0,1.0,3.0,4.0,7.0
574164,0.0,0,4.0,6.0,83.0,2.0,0,1.0,1.0,2.0,1.0,0,1.0,4.0,1.0,2.0,3.0,4.0,2.0,2.0,8.0
576638,0.0,0,2.0,7.0,1.0,3.0,0,20.0,1.0,NaN,4.0,1,1.0,2.0,2.0,1.0,2.0,3.0,3.0,4.0,7.0
577351,1.0,0,2.0,6.0,22.0,3.0,0,20.0,1.0,4.0,3.0,0,1.0,6.0,4.0,1.0,3.0,1.0,NaN,5.0,8.0


In [45]:
# Save train dataset 
train_data_small.to_csv('train_small.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train_small/train_small.csv')).upload_file('train_small.csv')

In [46]:
# Make a subset of the validation dataset (with only 20 predictors)
validation_data_small = validation_data[variables_to_keep]
print(validation_data_small.shape)
validation_data_small.head()

(683, 21)


,MATH_Proficient,LANGN_999,ST263Q02JA,ST259Q01JA,SC211Q03JA,ST251Q03JA,LANGN_922,ST059Q02JA,TARDYSD,ST290Q07WA,ST251Q04JA,LANGN_156,ST004D01T,IC173Q04JA,ST268Q04JA,ST251Q06JA,ST255Q01JA,IC178Q07JA,ST290Q05WA,IC174Q02JA,ST253Q01JA
574478,1.0,0,2.0,7.0,79.0,4.0,0,3.0,0.0,NaN,4.0,0,1.0,6.0,4.0,4.0,4.0,2.0,4.0,5.0,8.0
573678,1.0,0,2.0,9.0,22.0,3.0,0,7.0,1.0,NaN,3.0,0,2.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,5.0
576927,0.0,0,3.0,5.0,83.0,3.0,0,4.0,1.0,NaN,3.0,0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,NaN,5.0
576321,1.0,0,2.0,8.0,0.0,4.0,0,20.0,0.0,NaN,4.0,0,1.0,6.0,1.0,4.0,2.0,3.0,3.0,5.0,8.0
576339,1.0,0,3.0,4.0,30.0,2.0,0,7.0,0.0,NaN,2.0,0,2.0,NaN,4.0,1.0,2.0,NaN,3.0,NaN,7.0


In [47]:
# Save validation dataset 
validation_data_small.to_csv('validation_small.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation_small/validation_small.csv')).upload_file('validation_small.csv')

#### Train the model using the hyperparameters from the best model

In [48]:
# cell 15
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [49]:
# cell 16
s3_input_train_small = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train_small'.format(bucket, prefix), content_type='csv')
s3_input_validation_small = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation_small/'.format(bucket, prefix), content_type='csv')

In [50]:
# cell 17
sess = sagemaker.Session()

xgb_small = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

xgb_small.set_hyperparameters(# seed=42,       # Random seed (turned off because we are using a different seed per iteration)  
                        seed_per_iteration=True,   # Different seed for each boosting iteration, can prevent overfitting
                        early_stopping_rounds=10,   # Stop if AUC doesn’t improve for 10 rounds
                        # scale_pos_weight=imbalance_ratio, # Helps when outcome is imbalanced (but specificity decreased)
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=best_num_round,   # Number of boosting rounds for training                    
                        eta=best_eta,   # Learning rate, lower value is more robust to overfitting but requires more boosting rounds
                        max_depth=best_max_depth,   # Deeper trees can model more complex patterns but may overfit
                        min_child_weight=best_min_child_weight,   # Higher value ensures leaf nodes have sufficient samples, preventing overfitting
                        gamma=best_gamma,    # Higher values make it harder to partition a leaf node, making the algorithm more conservative
                        subsample=best_subsample,   # Fraction of training instances to use for each boosting round, < 1 can prevent overfitting
                        alpha=best_alpha   # L1 regularization term on weights, higher value leads to more regularization                       
                        )

xgb_small.fit({'train': s3_input_train_small, 'validation': s3_input_validation_small}) 

INFO:sagemaker:Creating training-job with name: xgboost-2025-02-27-06-17-25-742


2025-02-27 06:17:27 Starting - Starting the training job...
2025-02-27 06:17:41 Starting - Preparing the instances for training...
2025-02-27 06:18:02 Downloading - Downloading input data...
2025-02-27 06:18:33 Downloading - Downloading the training image...
2025-02-27 06:19:23 Training - Training image download completed. Training in progress...Arguments: train
[2025-02-27:06:19:38:INFO] Running standalone xgboost training.
[2025-02-27:06:19:38:INFO] File size need to be processed in the node: 0.27mb. Available memory size in the node: 8562.43mb
[2025-02-27:06:19:38:INFO] Determined delimiter of CSV input is ','
[06:19:38] S3DistributionType set as FullyReplicated
[06:19:38] 3186x20 matrix with 63422 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-02-27:06:19:38:INFO] Determined delimiter of CSV input is ','
[06:19:38] S3DistributionType set as FullyReplicated
[06:19:38] 683x20 matrix with 13582 entries loaded from /opt/ml/input/data/validation

## Deploy the model
Now that we've trained the `xgboost` algorithm on our data, let's deploy a model that's hosted behind a real-time endpoint.

In [51]:
test_data_small = test_data[variables_to_keep]

In [52]:
# cell 18
xgb_small_predictor = xgb_small.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2025-02-27-06-20-12-391
INFO:sagemaker:Creating endpoint-config with name xgboost-2025-02-27-06-20-12-391
INFO:sagemaker:Creating endpoint with name xgboost-2025-02-27-06-20-12-391


------!

In [53]:
# cell 19
xgb_small_predictor.serializer = sagemaker.serializers.CSVSerializer()

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batches to CSV string payloads (notice, we drop the target variable from our dataset first)
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [54]:
# Make predictions - Probabilities
predictions_small = predict(test_data_small.drop(['MATH_Proficient'], axis=1).to_numpy(), xgb_small_predictor)

In [55]:
# Clean up
xgb_small_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2025-02-27-06-20-12-391
INFO:sagemaker:Deleting endpoint with name: xgboost-2025-02-27-06-20-12-391


## Summary

#### Number of students not proficient in Math

In [56]:
#print("Students who are proficient: ", proficient_n)
print("Students who are NOT proficient in Math: ", not_proficient_n, "(", not_proficient_p, "%)")

Students who are NOT proficient in Math:  1607 ( 35.3 %)


#### Hyperparameters

In [57]:
print("seed_per_iteration: True")
print("early_stopping_rounds: 10")
print("objective: binary:logistic")
print("eval_metric: auc")
print("\nnum_round: ", best_num_round)
print("eta: ", round(best_eta, 2))
print("max_depth: ", best_max_depth)
print("min_child_weight: ", best_min_child_weight)
print("gamma: ", best_gamma)
print("subsample: ", round(best_subsample, 2))
print("alpha: ", best_alpha)


seed_per_iteration: True
early_stopping_rounds: 10
objective: binary:logistic
eval_metric: auc

num_round:  155
eta:  0.07
max_depth:  4
min_child_weight:  13
gamma:  4
subsample:  0.83
alpha:  1


#### Model performance (model with all the predictors)

***Adjust the threhold for the FINAL PREDICTIONS if necessary!!*** 

The model will predict as Math_proficient if the probability is above this threhold. (If the threshold is above 0.5, it will reduce the number of students predicted as "Math proficient" for both students that are actually proficient and not proficient in Math.)

In [58]:
threshold = 0.66

print("Threshold:", threshold)

Threshold: 0.66


In [59]:
cm = pd.crosstab(index=test_data['MATH_Proficient'], 
                 columns=np.round( (predictions >= threshold).astype(int) ), 
                 rownames=['actuals'], 
                 colnames=['predictions'])

TN = cm.loc[0.0, 0.0]
FP = cm.loc[0.0, 1.0]
FN = cm.loc[1.0, 0.0]
TP = cm.loc[1.0, 1.0]

accuracy = (TP + TN) / (TP + TN + FP + FN) * 100
precision = TP / (TP + FP) * 100 if (TP + FP) > 0 else 0
recall = TP / (TP + FN) * 100 if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
specificity = TN / (TN + FP) * 100 if (TN + FP) > 0 else 0

print(cm)
print("\nAccuracy (All features): {:.1f}".format(accuracy))
print("F1 Score (All features): {:.1f}".format(f1_score))
print("Precision (All features): {:.1f}".format(precision))
print("Recall (All features): {:.1f}".format(recall))
print("Specificity (All features): {:.1f}".format(specificity))

predictions    0    1
actuals              
0.0          189   37
1.0           75  382

Accuracy (All features): 83.6
F1 Score (All features): 87.2
Precision (All features): 91.2
Recall (All features): 83.6
Specificity (All features): 83.6


### Model performance (model with 20 predictors)

In [60]:
cm_small = pd.crosstab(index=test_data_small['MATH_Proficient'], 
                 columns=np.round( (predictions_small >= threshold).astype(int) ), 
                 rownames=['actuals'], 
                 colnames=['predictions'])

TN_small = cm_small.loc[0.0, 0.0]
FP_small = cm_small.loc[0.0, 1.0]
FN_small = cm_small.loc[1.0, 0.0]
TP_small = cm_small.loc[1.0, 1.0]

accuracy_small = (TP_small + TN_small) / (TP_small + TN_small + FP_small + FN_small) * 100
precision_small = TP_small / (TP_small + FP_small) * 100 if (TP_small + FP_small) > 0 else 0
recall_small = TP_small / (TP_small + FN_small) * 100 if (TP_small + FN_small) > 0 else 0
f1_score_small = 2 * (precision_small * recall_small) / (precision_small + recall_small) if (precision_small + recall_small) > 0 else 0
specificity_small = TN_small / (TN_small + FP_small) * 100 if (TN_small + FP_small) > 0 else 0

print(cm_small)
print("\nAccuracy (20 features): {:.1f}".format(accuracy_small))
print("F1 Score (20 features): {:.1f}".format(f1_score_small))
print("Precision (20 features): {:.1f}".format(precision_small))
print("Recall (20 features): {:.1f}".format(recall_small))
print("Specificity (20 features): {:.1f}".format(specificity_small))

predictions    0    1
actuals              
0.0          186   40
1.0           97  360

Accuracy (20 features): 79.9
F1 Score (20 features): 84.0
Precision (20 features): 90.0
Recall (20 features): 78.8
Specificity (20 features): 82.3


#### Top 20 features

In [61]:
pd.set_option('display.max_colwidth', None)
from IPython.display import display, Markdown

# Filter the DataFrame to only include rows where Variable_name is in top_20_features
top_20_dictionary = dictionary[dictionary["Variable_name"].isin(top_20_features)]
top_20_table = top_20_dictionary.set_index("Variable_name").loc[top_20_features].reset_index()
display(Markdown(top_20_table.to_markdown()))

KeyError: "['LANGN_999', 'ST263Q02JA', 'ST251Q03JA', 'ST290Q07WA', 'ST251Q04JA', 'IC173Q04JA', 'ST251Q06JA', 'ST255Q01JA', 'IC178Q07JA', 'ST290Q05WA', 'IC174Q02JA', 'ST253Q01JA'] not in index"